# *Required Libraries*

In [33]:
import spacy
import json
import pickle
from spacy.util import filter_spans
from spacy.tokens import DocBin
from tqdm import tqdm

# *Loading Data*

In [34]:
train_data=pickle.load(open("data\\train\\train_data.pkl","rb"))

In [35]:
with open('Corona2.json','r') as f:
    data=json.load(f)

In [36]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [37]:
temp_data = []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  temp_data.append(temp_dict)
print(temp_data[0])

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

In [38]:
for training_example in tqdm(temp_data):
    print('\nIteration\n')
    # print(training_example)
    # text = training_example['text']
    print(training_example['entities'])
    

100%|██████████| 31/31 [00:00<00:00, 31039.25it/s]


Iteration

[(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679, 689, 'MEDICINE'), (6, 23, 'MEDICINE'), (25, 37, 'MEDICINE'), (461, 470, 'MEDICALCONDITION'), (577, 589, 'MEDICINE'), (853, 865, 'MEDICALCONDITION'), (188, 198, 'MEDICINE'), (754, 762, 'MEDICALCONDITION'), (870, 880, 'MEDICALCONDITION'), (823, 833, 'MEDICINE'), (852, 853, 'MEDICALCONDITION'), (461, 469, 'MEDICALCONDITION'), (535, 543, 'MEDICALCONDITION'), (692, 704, 'MEDICINE'), (563, 571, 'MEDICALCONDITION')]

Iteration

[(364, 382, 'MEDICALCONDITION'), (0, 8, 'MEDICALCONDITION'), (94, 116, 'MEDICALCONDITION'), (178, 189, 'MEDICALCONDITION'), (221, 232, 'MEDICALCONDITION'), (23, 32, 'MEDICALCONDITION'), (409, 435, 'MEDICALCONDITION'), (386, 401, 'MEDICALCONDITION')]

Iteration

[(0, 22, 'MEDICINE'), (24, 27, 'MEDICINE'), (120, 123, 'MEDICINE'), (211, 214, 'PATHOGEN'), (52, 55, 'PATHOGEN'), (234, 237, 'MEDICINE'), (148, 151, 'PATHOGEN')]

Iteration

[(1211, 1589, 'MEDICALCONDITION'), (38, 

# *Building the Model*

In [39]:
for training_example in tqdm(train_data):
    print('\nIteration\n')
    print(training_example[1]['entities'])

100%|██████████| 200/200 [00:00<00:00, 275578.45it/s]


Iteration

[(1749, 1755, 'Companies worked at'), (1696, 1702, 'Companies worked at'), (1417, 1423, 'Companies worked at'), (1356, 1793, 'Skills'), (1209, 1215, 'Companies worked at'), (1136, 1248, 'Skills'), (928, 932, 'Graduation Year'), (858, 889, 'College Name'), (821, 856, 'Degree'), (787, 791, 'Graduation Year'), (744, 750, 'Companies worked at'), (722, 742, 'Designation'), (658, 664, 'Companies worked at'), (640, 656, 'Designation'), (574, 580, 'Companies worked at'), (555, 573, 'Designation'), (470, 493, 'Companies worked at'), (444, 469, 'Designation'), (308, 314, 'Companies worked at'), (234, 240, 'Companies worked at'), (175, 198, 'Companies worked at'), (93, 137, 'Email Address'), (39, 48, 'Location'), (13, 38, 'Designation'), (0, 12, 'Name')]

Iteration

[(2275, 2281, 'Companies worked at'), (2235, 2241, 'Companies worked at'), (1603, 1609, 'Companies worked at'), (667, 703, 'Skills'), (638, 658, 'College Name'), (612, 637, 'Degree'), (591, 611, 'College Name'), (587, 590,

In [40]:
# Initialize a blank English NLP model
nlp = spacy.blank('en')
doc_bin = DocBin()

for training_example in tqdm(train_data):
    text = training_example[0]
    labels = training_example[1]['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
doc_bin.to_disk("train.spacy")


 12%|█▏        | 24/200 [00:00<00:01, 111.46it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 25%|██▌       | 50/200 [00:00<00:01, 118.41it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 36%|███▋      | 73/200 [00:00<00:01, 107.54it/s]

...

 73%|███████▎  | 146/200 [00:01<00:00, 147.63it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 88%|████████▊ | 177/200 [00:01<00:00, 127.45it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 200/200 [00:01<00:00, 128.07it/s]


In [41]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [42]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
⚠ Aborting and saving the final best model. Encountered exception:
ValueError("[E024] Could not find an optimal move to supervise the parser.
Usually, this means that the model can't be updated in a way that's valid and
satisfies the correct annotations specified in the GoldParse. For example, are
all labels added to the model? If you're training a named entity recognizer,
also make sure that none of your annotated entity spans have leading or trailing
whitespace or punctuation. You can also use the `debug data` command to validate
your JSON-formatted training data. For det

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\cli\_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 723, in main
    return _main(
           ^^^^^^
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 193, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Users\M